# Requirement
gpu は on にして

+add Dataset の内容
- pytorch ignite 2.0
- word2vec google
- glove 840B 300d

In [ ]:
!pip install ../input/pytorch-ignite-020/pytorch_ignite-0.2.0-py2.py3-none-any.whl

In [ ]:
from collections import defaultdict
import gensim
from ignite.contrib.handlers.tensorboard_logger import *
from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Loss, RunningAverage, Precision, Recall
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import ProgressBar
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import pprint
import random
import re
from sklearn.model_selection import StratifiedKFold, train_test_split
%load_ext tensorboard.notebook
import tensorflow as tf
import time
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataset import Subset
import torchtext
from tqdm import tqdm_notebook as tqdm

# .The などが分かれないので，とりあえず .と大文字に対する分割
prog = re.compile('\.[A-Z]')
# 先頭大文字用
Head_check = re.compile('^[A-Z][^A-Z]+$')
# 乱数の固定
SEED = 0
# word2vec を学習するか
train_word2vec = False
# text の前処理をするか
text_preprocessing = True
# tensorboad 用の path
log_path = os.path.join("experiments", "tb_logs")
# best model 用の path
best_model_path = "best_model"
# MIN_COUNT に到達しない単語にも情報があれば付与したり，UNDEFINED などにしたほうが後々楽
MIN_COUNT = 10
debug = True
input_dir = os.path.join("..", "input")
jigsaw_path = "jigsaw-unintended-bias-in-toxicity-classification"

In [ ]:
print("check cuda", torch.cuda.is_available())
device = 'cuda'

In [ ]:
if debug:
    print("files in input directory")
    print("default path", os.listdir())
    print(input_dir, os.listdir(input_dir))
    for path in os.listdir(input_dir):
        print(os.path.join(input_dir, path), os.listdir(os.path.join(input_dir, path)))

In [ ]:
# disable progress bars when submitting らしいです．よく知らないです
# kaggle 関係は別の人に聞いてください
def is_interactive():
    #print("os environment", 'SHLVL' not in os.environ)
    return 'SHLVL' not in os.environ

# 環境によって，tqdm が動作しないようですが，今回使ってないです
if not is_interactive():
    def nop(it, *a, **k):
        return it

    tqdm = nop

In [ ]:
# seed を固定しないと，実行結果が毎回変わってしまいます
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

# data の確認
特に調べずに preprocessing しちゃっていますが，悪意のある単語というのはある程度決まっていると思うので，確認した方が良いと思います．

例えばくらいですが
https://www.kaggle.com/nz0722/simple-eda-text-preprocessing-jigsaw

In [ ]:
train_df = pd.read_csv(os.path.join(input_dir, jigsaw_path, "train.csv"))
if debug:
    print("train columns", train_df.columns)
    print("train shape", train_df.shape)
train_df.head()

In [ ]:
test_df = pd.read_csv(os.path.join(input_dir, jigsaw_path, "test.csv"))
if debug:
    print("test columns", test_df.columns)
    print("test shape", test_df.shape)
test_df.head()

In [ ]:
test_df.tail()

In [ ]:
if train_word2vec:
    target = np.where(train_df["target"] >= 0.5, True, False)
train_shape = train_df.shape
all_document = train_df["comment_text"].tolist()+test_df["comment_text"].tolist()
# kaggle だとメモリ足らない
del train_df, test_df

In [ ]:
# list の混在がないかチェック
if debug:
    length = 2
    print(all_document[:length])
    print(all_document[train_shape[0]-length:train_shape[0]])
    print(all_document[train_shape[0]:train_shape[0]+length])
    print(all_document[-length:])

# Text preprocessing
stanfordnlp package が最も精度が良いとは思いますが，kaggle の kernel 上では厳しいなって感じです．
ローカルでやったのを上げて使えるようですが，詳しくないので．特に，単語間の依存関係は使えるかもしれません

nltk や stanfordnlp のほかに，spacy や sentencepiece もあります．また，keras や torch にも組み込みのものはあるような．

In [ ]:
# 先頭大文字への対処．固有名詞には基本触れない
lemmatizer = nltk.stem.WordNetLemmatizer()
def word_compile(word, lemmatize = True):
    # 先頭のみ大文字
    check = Head_check.match(word)
    if check is not None:
        word = word[0].lower() + word[1:]
    # 見出し語化
    if lemmatize:
        word = lemmatizer.lemmatize(word)
    return word

if debug:
    print(word_compile("They"))
    print(word_compile("TRUMP"))
    print(word_compile("WordNet"))

In [ ]:
# .が繋がっているとわかれないので
def splitter(text, split_words, include = True):
    split_words = [i.span()[0] for i in split_words]
    word_list = []
    word = ""
    for i in range(len(text)):
        word += text[i]
        if i in split_words:
            word_list.append(word_compile(word[:-1]))
            if include:
                word_list.append(".")
            word = ""
    word_list.append(word_compile(word))
    return word_list

if debug:
    print(splitter("borrow.Incoming", prog.finditer("borrow.Incoming")))

In [ ]:
# sentence.split() はnltk.word_torknize(sentence) でも可
def tokenizer(sentence):
    global text_preprocessing
    if text_preprocessing:
        return [word_compile(word) if len(prog.findall(word)) == 0 else splitter(word, prog.finditer(word))[0] 
                      for word in nltk.word_tokenize(sentence)]
    else:
        # どっかで数字だけの行があるぽいので，雑に str で処理しました
        return sentence.split()

sentences = [tokenizer(sentence) for sentence in all_document]
if debug:
    print(sentences[:2])
del all_document

In [ ]:
# 出現回数の少ない順にソート（確認のため）
word_count = defaultdict(lambda: 0)
if text_preprocessing:
    for sentence in sentences:
        for word in sentence:
            word_count[word] += 1
    word_count = dict(sorted(word_count.items(), key = lambda kv: kv[1]))
if debug:
    print(word_count)

# word embedding

In [ ]:
# 面倒だったので､やっつけ仕事
class emb_model(object):
    def __init__(self, vectors, index2word):
        self.vectors = vectors
        self.vector_size = vectors.shape[1]
        self.index2word = index2word

encoding = "utf8"
#def float_decode(value):
#    return float(value.decode(encoding))

def load_glove(file_path, word_candidates):
    vectors = []
    index2word = []
    with open(file_path, "rb") as f:
        for line in f:
            #print(line)
            sline = line.split()
            word, vector = sline[0].decode(encoding), np.asarray(sline[1:], dtype=np.float32)
            if vector.shape[0] != 300:
                print(line)
                break
            if word_candidates is None or word in word_candidates:
                vectors.append(vector)
                index2word.append(word)
    model = emb_model(np.asarray(vectors, dtype=np.float32), index2word)
    return model

In [ ]:
if train_word2vec:
    word2vec = gensim.models.Word2Vec(sentences, size=300, window=5, min_count=MIN_COUNT , iter=5, negative=5)
else:
    # word2vec でも学習したコーパスによって特性が変わる恐れがある
    # これは Google News を学習したモデル．今回のテキストを学習したモデルも誰かがあげてくれているみたいです．
    # 有名な単語分散表現学習モデルは word2vec, glove, fasttext
    # 理解としては，word2vec がノーマル．glove がドメイン特化，fasttext が未知語に強いくらいです
    # fasttext は未知語に強いですが，pre や con などの接頭辞・接尾辞にように同じ特徴を持つ場合に強いので
    # FUUUUUUU などは確定で強いかはやってみてという感じ
    # gloveとfasttextとcharngram は torch から直接呼び出せます
    
    word_candidates = set([word for sentence in sentences for word in sentence])
    # 正直やりたくないが，メモリのためには仕方ない
    del sentences
    
    # model なら word2vec = gensim.models.Word2Vec.load("")　今回は bin 形式なので
    word2vec = gensim.models.KeyedVectors.load_word2vec_format(
        os.path.join(input_dir, "word2vec-google", "GoogleNews-vectors-negative300.bin"), binary = True)
    # 変数名はすいません・・・
    # word2vec = load_glove(os.path.join(input_dir, "glove840b300dtxt", "glove.840B.300d.txt"), word_candidates)

In [ ]:
# emb_model は非対応
if not isinstance(word2vec, emb_model):
    print("check original index", word2vec.vocab["suck"].index)
    if not train_word2vec:
        # 拾ってきた分散表現は基本的に無駄にメモリを取るので，削ります　※バグを生みやすいのでお勧めはしません
        # とりあえず，今回は雑に削ります．
        index_candidates = [i for i, word in enumerate(word2vec.index2word) if word in word_candidates]
        word2vec.vectors = word2vec.vectors[index_candidates]
        word2vec.index2word = [word2vec.index2word[i] for i in index_candidates]

        vocab_dict = {}
        for i, word in enumerate(word2vec.index2word):
            vocab = word2vec.vocab[word]
            vocab.index = i
            vocab_dict[word] = vocab
        del word_candidates
    print("check revise index", word2vec.vocab["suck"].index)
    
    if debug:
        print(word2vec.most_similar("suck"))

# データの用意
すいませんが，validation までを用意します．
予測までは追々

メモリに既に展開済みかで対応を分けています

In [ ]:
if train_word2vec:
    TEXT = torchtext.data.Field(sequential=True, include_lengths=False,
                               batch_first=True, tokenize=tokenizer)
    # 見た感じ，2値分類にしている例が多いので
    TARGET = torchtext.data.LabelField(sequential=False, use_vocab=False,
                                      batch_first=True, is_target=True, 
                             preprocessing=lambda x: int(float(x) >= 0.5))
    fields=[('text', TEXT), ('target', TARGET)]

    # 今回はリストに既に展開しているので Example.fromlist と Dataset
    examples = [torchtext.data.Example.fromlist(
                [sentence, target], fields=fields) 
                for sentence, target in zip(sentences, target)]

    train_data = torchtext.data.Dataset(
         examples, fields=fields)
        
else:
    TEXT = torchtext.data.Field(sequential=True, include_lengths=False,
                               batch_first=True, tokenize=tokenizer)
    # 見た感じ，2値分類にしている例が多いので
    TARGET = torchtext.data.LabelField(sequential=False, use_vocab=False,
                                      batch_first=True, is_target=True, 
                             preprocessing=lambda x: int(float(x) >= 0.5))
    fields={"comment_text":('text', TEXT), 
            "target":('target', TARGET)}
    
    train_data = torchtext.data.TabularDataset(
        path = os.path.join(input_dir, jigsaw_path, "train.csv"),
        format="csv",     
        fields=fields,
        skip_header=False)
    
if debug:
    print(train_data.examples[0].text)
    print(train_data.examples[0].target)

In [ ]:
# 面倒なので新しいのを作りました
# 普通は vectors から読み込めます
class word_vectors(torchtext.vocab.Vectors):
    def __init__(self, model, unk_init=None):
        self.itos = model.index2word
        self.stoi = {word:i for i, word in enumerate(model.index2word)}
        self.vectors = torch.FloatTensor(model.vectors)
        self.dim = model.vector_size
        self.unk_init = torch.Tensor.zero_ if unk_init is None else unk_init
        # self.cache(name, cache, url=url, max_vectors=max_vectors)
# 今回の word2vec 用のベクトルを torch で使えるようにしてます        
vectors = word_vectors(word2vec)    

In [ ]:
# 単語を index 形式にして，出現頻度が低い単語をまとめてます
TEXT.build_vocab(train_data, min_freq=MIN_COUNT, vectors=vectors)
if debug:
    print(TEXT.vocab.freqs.most_common(100))

In [ ]:
if debug:
    print(TEXT.vocab.itos[:100])
    # 元の値は 0.89 なので 2値化できているのを確認
    print(train_data.examples[4].target)

In [ ]:
# submit の input データも用意
batch_test_size = 1024
submit_dataset = torchtext.data.TabularDataset(os.path.join(input_dir, jigsaw_path, "test.csv"),
                                            format='csv',
                                            fields={'comment_text': ('text', TEXT)})
submit_loader = torchtext.data.Iterator(submit_dataset, batch_size=batch_test_size,
                                      device='cuda', shuffle=False, sort=False)

# Trainer の用意
ここは､どのような学習手法でも変わらないことが多いため使いまわせるものを自前で持っていた方が良い

今回は高級なものを使用
（こういったパッケージを使うと細かいところに手が届かなかったり､バージョン更新が面倒だったりする）

その結果として，逆にコードが増えた．非常によろしくないと思う．

# Trainer のイメージ
普通に省略しすぎて，Train のイメージすら消してしまったので・・・

位置は前後して良いですが､今回の場合

- model の指定
- optimizer の指定

- STARTED    
    - EPOCH STARTED
    
        cross validation によるデータの指定
        
        - ITERATION STARTED
            
            process_function（train に対する学習）            
            
        - ITERATION COMPLETED
        
   eval_function（train, validation 等の評価）
        
    - EPOCH COMPLETED
- COMPLETED

get_prediction（submit 用の予測）

In [ ]:
# 誤差関数の指定
criterion = nn.CrossEntropyLoss()

In [ ]:
# 学習時
def process_function(engine, batch):
    # 学習用
    model.train()
    # 勾配の初期化
    optimizer.zero_grad()
    # データの読み込み
    x, y = batch.text.cuda(), batch.target.cuda()
    # 順伝播
    y_pred = model(x)
    # 誤差の算出
    loss = criterion(y_pred, y)
    # 勾配の計算
    loss.backward()
    # パラメータの更新
    optimizer.step()
    return loss.item()

In [ ]:
# 評価時
def eval_function(engine, batch):
    # 評価用（一部の層では学習時と評価時で動作が変わるため）
    model.eval()
    with torch.no_grad():
        x, y = batch.text.cuda(), batch.target.cuda()
        y_pred = model(x)
        return y_pred, y

In [ ]:
# 予測用
# 予測用では，引数がmodel であったり
# ソフトマックスにかけている所が違います
def get_predictions(model, loader):
    model.eval()
    with torch.no_grad():
        predictions = []
        for batch in loader:
            x = batch.text.cuda()
            logits = model(x)
            y_pred = F.softmax(logits, dim=1)[:, 1]
            # move from GPU to CPU and convert to numpy array
            # cpu で計算できる方にする必要があります
            y_pred_numpy = y_pred.cpu().numpy()

            predictions.append(y_pred_numpy)
        predictions = np.concatenate(predictions)
    return predictions

In [ ]:
# 今回はignite の Engine を使ったので､学習部分と評価部分を定めます
trainer = Engine(process_function)
train_evaluator = Engine(eval_function)
validation_evaluator = Engine(eval_function)
test_evaluator = Engine(eval_function)

In [ ]:
# ここら辺が逆に不便なのですが､metrics を追加していきます
# 追加method.attach(追加engine, metric名)
RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

# 小数点をカットするだけなので､なくても可
def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(y_pred)
    return y_pred, y
# accurracy, loss の追加
Accuracy(output_transform=thresholded_output_transform).attach(
    train_evaluator, 'accuracy')
Loss(criterion).attach(train_evaluator, 'ce')
Accuracy(output_transform=thresholded_output_transform).attach(
    validation_evaluator, 'accuracy')
Loss(criterion).attach(validation_evaluator, 'ce')
Accuracy(output_transform=thresholded_output_transform).attach(
    test_evaluator, 'accuracy')
Loss(criterion).attach(test_evaluator, 'ce')

In [ ]:
# 内部で新しい tqdm が入れられるが，自前で定義済みなので
# 定義してなければ不要
class CustomProgressBar(ProgressBar):
    def __init__(self, tqdm, persist=False,
                 bar_format='{desc}[{n_fmt}/{total_fmt}] {percentage:3.0f}%|{bar}{postfix} [{elapsed}<{remaining}]',
                 **tqdm_kwargs):
        self.pbar_cls = tqdm
        self.pbar = None
        self.persist = persist
        self.bar_format = bar_format
        self.tqdm_kwargs = tqdm_kwargs

In [ ]:
# 学習時に表示するものやログに残すものを指定します

# tensorboard 用の logger
# 必要な metrics を指定してます
tb_logger = TensorboardLogger(log_dir=log_path)
tb_logger.attach(trainer,
                 log_handler=OutputHandler(tag="training", output_transform=lambda loss: {'loss': loss}),
                 event_name=Events.ITERATION_COMPLETED)
tb_logger.attach(train_evaluator,
                 log_handler=OutputHandler(tag="training",
                                              metric_names=["ce", "accuracy"],
                                              another_engine=trainer),
                 event_name=Events.EPOCH_COMPLETED)
tb_logger.attach(validation_evaluator,
                 log_handler=OutputHandler(tag="validation",
                                           metric_names=["ce", "accuracy"],
                                           another_engine=trainer),
                 event_name=Events.EPOCH_COMPLETED)
tb_logger.attach(test_evaluator,
                 log_handler=OutputHandler(tag="test",
                                           metric_names=["ce", "accuracy"],
                                           another_engine=trainer),
                 event_name=Events.EPOCH_COMPLETED)

# tqdm 用の logger 
# submit では使えない
if is_interactive():
    pbar = CustomProgressBar(tqdm=tqdm, persist=True, bar_format="")
    pbar.attach(trainer, ['loss'])
    
def plot_metrics(engine, metrics):
    if is_interactive():
        pbar.log_message(
            "Training Results - Epoch: {} \nMetrics\n{}"
            .format(engine.state.epoch, pprint.pformat(metrics)))
    else:
        print("Training Results - Epoch: {} \nMetrics\n{}"
            .format(engine.state.epoch, pprint.pformat(metrics)))
    
def log_training_results(engine):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    plot_metrics(engine, metrics)

def log_validation_results(engine):
    validation_evaluator.run(val_loader)
    metrics = validation_evaluator.state.metrics
    plot_metrics(engine, metrics)
        
def log_test_results(engine):
    test_evaluator.run(test_loader)
    metrics = test_evaluator.state.metrics
    plot_metrics(engine, metrics)

trainer.add_event_handler(Events.EPOCH_COMPLETED, log_training_results)
trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)
trainer.add_event_handler(Events.EPOCH_COMPLETED, log_test_results)

In [ ]:
# 正直，いらなくね．と思いましたが，一応残しときます
def score_function(engine):
    val_loss = engine.state.metrics['ce']
    return -val_loss

# score function によってモデルを評価してます（loss の負の値なので，lossの小さい順）
# patient は loss が何回（連続？）で良くならなかったかを我慢する数
handler = EarlyStopping(patience=3, score_function=score_function, trainer=trainer)

test_evaluator.add_event_handler(Events.COMPLETED, handler)

In [ ]:
# best モデルの save
best_model_save = ModelCheckpoint(
    best_model_path, 'BiLSTM', n_saved=1,
    create_dir=True, save_as_state_dict=True,
    score_function=score_function)

# cross validation
2重のバリデーション（nested cross validation?）になっている

### これだと，データ全部を使うわけではないので注意

In [ ]:
batch_size = 1024
test_size = 0.1
# kfold の分割数
n_folds = 10

In [ ]:
# torchtext は普通の index で回すのに向いてないので（不便すぎる）
# kflold に合わせるための変更
class TorchtextSubset(Subset):
    def __init__(self, dataset, indices):
        super(TorchtextSubset, self).__init__(dataset, indices)
        self.fields = self.dataset.fields
        self.sort_key = self.dataset.sort_key

In [ ]:
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
labels = [example.target for example in train_data]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(range(len(train_data)), labels, test_size=test_size, random_state=SEED)
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
idx_splits = list(skf.split(X_train, y=y_train))
if debug:
    print(len(idx_splits))

In [ ]:
# cross validation にするために，epoch 毎に index を変えている
@trainer.on(Events.EPOCH_STARTED)
def cross_validation(engine):
    # loader も global 変数で回るので global 変数にしなければいけない（不便）
    global train_loader, val_loader
    train_idx, val_idx = idx_splits[engine.state.epoch-1 % n_folds]
    train_ds = TorchtextSubset(train_data, train_idx)
    val_ds = TorchtextSubset(train_data, val_idx)

    # データ長を揃えてくれる（sort key で指定）
    train_loader, val_loader = torchtext.data.BucketIterator.splits(
        [train_ds, val_ds], batch_sizes=[batch_size, batch_size], device=device,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True, repeat=False)
    engine.state.dataloader = train_loader
    
test_idx = X_test
test_ds = TorchtextSubset(train_data, test_idx)
test_loader = torchtext.data.BucketIterator(dataset=test_ds, batch_size=batch_size,
                                            device=device, sort_key=lambda x: len(x.text),
                                            sort_within_batch=True, repeat=False)

# model の用意
ちょっと時間的にすぐ終わらなそうなので，モデルは借りました
https://www.kaggle.com/protan/lstm-cnn-torchtext-with-ignite

※ただし，アンサンブルと target 以外の予測は簡単化のため消してます

時系列のモデルとしては，RNN, LSTM, GRU が有名です．
※ Transformer というモデルが言語処理では最も強いと言われていますが，上記と並べにくいので省いています
- RNN は simple な時系列に対応したネットワークで，時系列長が長いと学習に失敗しやすいです
- LSTM は Long Short Term Memory の略称で時系列で離れた単語や近い単語を強調したりする仕組みが追加されています
- GRU は LSTM の枠組みをシンプルにしたもので，ネットワークのパラメータが LSTM より少なく時系列長に対して RNN より頑強です

RNN と LSTM
https://qiita.com/KojiOhki/items/89cd7b69a8a6239d67ca

LSTMのみならこれが一番だと思います
https://qiita.com/t_Signull/items/21b82be280b46f467d1b

GRU はこれらのモデルがわかっていれば概要図か数式を見ればすぐわかるかと思います

純粋な拡張として，attention+RNN があります

その他にも kernel で見られるように単語分散表現以外にも単語のベクトルが得られる手法があり，それらが，ELMO, open-NMT, BERT というもので pretrained のものを興味があれば使えば良いと思います．
（学習時間的に基本無理です）

In [ ]:
# 学習用パラメーター
# CNN 用
kernel_sizes = [3, 4, 5]
num_filters = 64
# クラス数
num_classes = 2
# embedding の学習の有無
mode = 'nonstatic' #'static' or 'nonstatic'
hidden_dim = 256
lstm_units = 128
# spatial dropout をする確率
spatial_drop = 0.1
# 通常の dropout をする確率
d_prob = 0.5

In [ ]:
# Dropout は BatchNormalization 同様有名です
# Dropout は確率で選択した一部のノードを使用しない方法で過学習を避けやすい
# らしいです．元の論文を確かめてないです．
# Spatial は Dropout に比べて，連続的に落とします
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, kernel_sizes, num_filters,
                 num_classes, d_prob, mode, hidden_dim, lstm_units,
                 emb_vectors=None, spatial_drop=0.1):
        super(BiLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        #self.kernel_sizes = kernel_sizes
        #self.num_filters = num_filters
        self.num_classes = num_classes
        self.d_prob = d_prob
        self.mode = mode
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
        self.embedding_dropout = SpatialDropout(spatial_drop)
        
        # embedding の読み込みと学習するかどうかの確認
        if emb_vectors is not None:
            self.load_embeddings(emb_vectors)
        # CNN は絶対じゃないので
        """
        self.conv = nn.ModuleList([nn.Conv1d(in_channels=embedding_dim,
                                             out_channels=num_filters,
                                             kernel_size=k, stride=1) for k in kernel_sizes])
        """
        # 1個目のBiLSTM
        self.lstm1 = nn.LSTM(embedding_dim, lstm_units,
                             bidirectional=True, batch_first=True)
        # 2個目のLSTM
        self.lstm2 = nn.LSTM(lstm_units * 2, lstm_units,
                             bidirectional=True, batch_first=True)
        #self.lstm_body = nn.LSTM(
        #    embedding_dim, lstm_units, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(d_prob)
        # remove CNN
        #self.fc = nn.Linear(len(kernel_sizes) * num_filters, hidden_dim)
        #self.fc_total = nn.Linear(hidden_dim * 1 + lstm_units * 4, hidden_dim)
        self.fc_total = nn.Linear(lstm_units * 4, hidden_dim)
        self.fc_final = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x_emb = self.embedding(x)
        x_emb = self.embedding_dropout(x_emb)
        
        # remove CNN
        # pad for CNN kernel 5
        """
        if x_emb.shape[1] < 5:
            x_emb = F.pad(x_emb, (0, 0, 0, 5 - x_emb.shape[1]), value=0)
        
        x = [F.relu(conv(x_emb.transpose(1, 2))) for conv in self.conv]
        x = [F.max_pool1d(c, c.size(-1)).squeeze(dim=-1) for c in x]
        x = torch.cat(x, dim=1)
        x = self.fc(self.dropout(x))
        """

        h_lstm1, _ = self.lstm1(x_emb)
        h_lstm2, _ = self.lstm2(h_lstm1)

        # average pooling
        avg_pool2 = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool2, _ = torch.max(h_lstm2, 1)
        
        # cat は concat（結合）です
        #out = torch.cat([x, avg_pool2, max_pool2], dim=1)
        out = torch.cat([avg_pool2, max_pool2], dim=1)
        # relu は活性化関数の一つです．これもいくつか種類があります
        out = F.relu(self.fc_total(self.dropout(out)))
        out = self.fc_final(out)

        return out

    # 参考にした人が丁寧な人だったので
    # embedding 層を学習するかしないかを選べるようにしてくれています
    # non static では最初に入れた embedding の重みも変わります
    # 転移学習したい時などに embedding を学習させます
    def load_embeddings(self, emb_vectors):
        if 'static' in self.mode:
            self.embedding.weight.data.copy_(emb_vectors)
            if 'non' not in self.mode:
                self.embedding.weight.data.requires_grad = False
                print('Loaded pretrained embeddings, weights are not trainable.')
            else:
                self.embedding.weight.data.requires_grad = True
                print('Loaded pretrained embeddings, weights are trainable.')
        elif self.mode == 'rand':
            print('Randomly initialized embeddings are used.')
        else:
            raise ValueError(
                'Unexpected value of mode. Please choose from static, nonstatic, rand.')

In [ ]:
# もっと高レベルなモデルの可視化の方法もあるので､調べるとよいと思います
# 今回は､最低限の確認です
vocab_size, embedding_dim = TEXT.vocab.vectors.shape
model = BiLSTM(vocab_size=vocab_size,
              embedding_dim=embedding_dim,
              kernel_sizes=kernel_sizes,
              num_filters=num_filters,
              num_classes=num_classes,
              d_prob=d_prob,
              mode=mode,
              hidden_dim=hidden_dim,
              lstm_units=lstm_units,
              spatial_drop=spatial_drop,
              emb_vectors=TEXT.vocab.vectors)
model

# 学習

In [ ]:
# model や optimizer は global 変数で共有しているよう（気持ち悪いけども）
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
test_evaluator.add_event_handler(Events.EPOCH_COMPLETED, best_model_save, {'text_model': model})

In [ ]:
trainer.run(None, max_epochs=10)

# tensorboard logger は閉じる必要がある
tb_logger.close()

# log の確認

In [ ]:
# 一応，ログも出しておいた
log = {"loss": {"training":[]},
       "accuracy": {"training":[], "validation":[], "test":[]},
       "ce": {"training":[], "validation":[], "test":[]}}
for event in tf.train.summary_iterator(os.path.join(log_path, os.listdir(log_path)[0])):
    for value in event.summary.value:
        if value.HasField('simple_value'):
            timing, metric = value.tag.split("/")
            log[metric][timing].append(value.simple_value)
            
for key, value in log.items():
    for k, v in value.items():
        plt.plot(np.arange(len(log[key][k])), log[key][k], label=k)
    plt.title(key)
    plt.legend()
    plt.show()

In [ ]:
# 提出時は使えない
%tensorboard --logdir experiments/tb_logs

# submit データに対する予測

In [ ]:
# load best model 
model_path = os.path.join(best_model_path, os.listdir(best_model_path)[0])
model_state_dict = torch.load(model_path)
model.load_state_dict(model_state_dict)

predictions = get_predictions(model, submit_loader)

# submit ファイルの作成

In [ ]:
# 提出ファイルを作成
df_sub = pd.read_csv(os.path.join(input_dir, jigsaw_path, "sample_submission.csv"))
df_sub['prediction'] = predictions

In [ ]:
df_sub.head()

In [ ]:
df_sub.to_csv('submission.csv', index=False)